In [ ]:
"""

Created on Tue Feb 11 18:03 2025

Prepare initial conditions for ice-shelf cavities for PISCES

Author: C. Burgard

"""

In [1]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
from cdo import Cdo
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


FUNCTIONS

In [4]:
def weighted_mean(data, dims, weights):
    weight_sum = weights.sum(dim=dims) # to avoid dividing by zero
    return (data*weights).sum(dim=dims)/weight_sum.where(weight_sum != 0)

READ IN DATA

In [71]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_interim2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_thredds = '/thredds/tgcc/work/burgardc/FORCING_FILES_TO_BE_EXTRAP/'
outputpath = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/PISCES_input/'

In [7]:
var_list = ['CO_PISCES_annual', 'COS_PISCES_annual', 'ISP_PISCES_annual', 'N2O_PISCES_annual',
             'DMS_PISCES_annual', 'NO3_WOA2013',  'Fer_FEMIP_model_median', 'DOC_PISCES_monthly', 
             'Si_WOA2013','PO4_WOA2013', 'O2_WOA2013', 'TALK_GLODAPv2_Lauvset2016', 
             'PiDIC_GLODAPv2_Lauvset2016']             #,'eORCA1.4.2_sali_ref_clim_monthly.nc'

Define the masks to make the mean

In [22]:
ds_isfNEMO= xr.open_dataset(inputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [23]:
mask_fronts = xr.open_dataset(inputpath_interim2 + 'mask_fronts_eORCA1.nc')

In [24]:
file_Justine = xr.open_dataset(inputpath_raw2 + 'Mask_Iceshelf_IMBIE2_v2.nc')

In [25]:
isfmask_nn = cdo.remapnn(inputpath_raw + 'griddes_latlon_1deg.txt',input=file_Justine['Iceshelf_extrap'], returnArray='Iceshelf_extrap')
isfmask_extrap_1deg = xr.DataArray(data=isfmask_nn, dims=['y','x'])

In [26]:
isfmask_nn = cdo.remapnn(inputpath_raw + 'griddes_latlon_1deg.txt',input=file_Justine['Iceshelf'], returnArray='Iceshelf')
isfmask_1deg = xr.DataArray(data=isfmask_nn, dims=['y','x'])

In [27]:
isfmask_nn_from_NEMO_tolatlon = xr.open_dataset(inputpath_interim + 'test_put_NEMO_on_1deg.nc')

Define the previously closed regions

In [28]:
landsea_mask = pd.read_csv('/home/cburgard/SCRIPTS/ISFinNEMO_Clara/WOA13_mask.csv',delimiter=',',header=1)

In [29]:
landsea_mask_xr = xr.DataArray(data=np.reshape(landsea_mask['Bottom_Standard_Level'], (180,360)), dims=['y','x']).assign_coords({'x': np.concatenate([np.arange(180,360),np.arange(0,180)]), 'y': np.arange(0,180)})#.assign_coords({'x': np.concatenate([np.arange(0.5,179.6),np.arange(-179.5,-0.4)]), 'y': np.arange(-89.5,89.6)})#.assign_coords({'x': np.arange(-179.5,179.6), 'lat': np.arange(-89.5,89.6)})
ls_mask_01 = landsea_mask_xr.sortby(landsea_mask_xr.x) == 1

Identify the regions in front of the ice shelves I want to open and average T and S

In [30]:
ls_mask_01.plot()

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [31]:
mask_0_1_2 = ls_mask_01.copy()
mask_0_1_2 = mask_0_1_2.where(mask_0_1_2 != 1,3)
mask_0_1_2 = mask_0_1_2.where(mask_0_1_2 != 0,1)

In [32]:
mask_front0 = mask_0_1_2.where(isfmask_1deg > 0,1).copy()

In [ ]:
mask_front.plot()

In [ ]:
isfmask_1deg.plot()

In [ ]:
(mask_front * 0 + isfmask_extrap_1deg).plot()

In [ ]:
ls_mask_01.where(mask_front == 5).plot()

In [33]:
mask_front = mask_front0.copy()
mask_front = mask_front.where((mask_front0.roll(x=-1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(x=1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(y=-1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(y=1)-mask_front0)!=2,5)
# cut out all front points
mask_front = mask_front.where(mask_front==5)

# set the ice shelf number
mask_front = mask_front * 0 + isfmask_extrap_1deg

MAKE THE VERTICAL MEAN PROFILES

In [34]:
cell_area = xr.open_dataset(inputpath_interim + 'gridarea_1deg.nc')

In [35]:
cell_area = cell_area.rename({'lat':'y','lon':'x'}).assign_coords({'x': np.arange(0,360), 'y': np.arange(0,180)})['cell_area']

In [43]:
mask_front

<xarray.DataArray (y: 180, x: 360)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [82., 82., 82., ..., 41., 41., 41.]])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 171 172 173 174 175 176 177 178 179

In [53]:
mask_front_latlon = mask_front.rename({'x': 'lon', 'y': 'lat'})
mask_front_latlon = mask_front_latlon.assign_coords({'lat': data_init_0.lat.values, 'lon': np.concatenate([np.arange(180,360),np.arange(0,180)])})
cell_area_latlon = cell_area.rename({'x': 'lon', 'y': 'lat'})
cell_area_latlon = cell_area_latlon.assign_coords({'lat': data_init_0.lat.values, 'lon': np.concatenate([np.arange(180,360),np.arange(0,180)])})

In [84]:
var_list = ['CO_PISCES_annual', 'COS_PISCES_annual', 'ISP_PISCES_annual', 'N2O_PISCES_annual',
             'DMS_PISCES_annual', 'NO3_WOA2013',  'Fer_FEMIP_model_median', 'DOC_PISCES_monthly', 
             'Si_WOA2013','PO4_WOA2013', 'O2_WOA2013', 'TALK_GLODAPv2_Lauvset2016', 
             'PiDIC_GLODAPv2_Lauvset2016']             #,'eORCA1.4.2_sali_ref_clim_monthly.nc'

var_list = [ 'NO3_WOA2013' 
             ]             #'NO3_WOA2013',  'Fer_FEMIP_model_median', 'Si_WOA2013''O2_WOA2013'

In [94]:
front_mask_kisf.squeeze()

<xarray.DataArray (lat: 180, lon: 360)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * lon      (lon) int64 180 181 182 183 184 185 186 ... 174 175 176 177 178 179
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
    ID       int64 1

In [85]:
for vv in var_list:
    print(vv)
    data_init_0 = xr.open_dataset(inputpath_thredds + vv+'_r360x180xl75.nc')[vv.split('_')[0]]

    vv_mean_list = []
    
    for ID in tqdm(file_Justine.ID):
        if ((mask_front_latlon == ID).sum() > 1) and (ID != 67) and (ID != 125):
            if ID == 66:
                front_mask_kisf = (mask_front_latlon == 66) | (mask_front_latlon == 67)
            elif ID == 124:
                front_mask_kisf = (mask_front_latlon == 124) | (mask_front_latlon == 125)
            else:
                front_mask_kisf = (mask_front_latlon == ID)
                
            vv_mean = weighted_mean(data_init_0.where(front_mask_kisf), ['lat','lon'], cell_area_latlon.where(front_mask_kisf))
            
            vv_mean_list.append(vv_mean.assign_coords({'ID': ID}))
    
    vv_mean_all = xr.concat(vv_mean_list, dim='ID')
    
    vv_mean_all.rename(vv).to_netcdf(outputpath + vv+'_r360x180xl75_withisfcav.nc')


NO3_WOA2013


  0%|          | 0/133 [00:00<?, ?it/s]

ValueError: size of dimension 'depth' on inputs was unexpectedly changed by applied function from 75 to 1. Only dimensions specified in ``exclude_dims`` with xarray.apply_ufunc are allowed to change size.

In [79]:
# this is on the NEMO grid so I need to recheck the stuff for the front
#'TALK_GLODAPv2_Lauvset2016''PiDIC_GLODAPv2_Lauvset2016'
data_init_0 = xr.open_dataset(inputpath_thredds + 'eORCA1.4.2_sali_ref_clim_monthly.nc')['vosaline']


vv_mean_list = []

for ID in tqdm(file_Justine.ID):
    if ((mask_front_latlon == ID).sum() > 1) and (ID != 67) and (ID != 125):
        if ID == 66:
            front_mask_kisf = (mask_front_latlon == 66) | (mask_front_latlon == 67)
        elif ID == 124:
            front_mask_kisf = (mask_front_latlon == 124) | (mask_front_latlon == 125)
        else:
            front_mask_kisf = (mask_front_latlon == ID)
            
        vv_mean = weighted_mean(data_init_0.where(front_mask_kisf), ['lat','lon'], cell_area_latlon.where(front_mask_kisf))
        
        vv_mean_list.append(vv_mean.assign_coords({'ID': ID}))

vv_mean_all = xr.concat(vv_mean_list, dim='ID')

vv_mean_all.rename('vosaline').to_netcdf(outputpath + 'eORCA1.4.2_sali_ref_clim_monthly_withisfcav.nc')


  0%|          | 0/133 [00:00<?, ?it/s]

MemoryError: Unable to allocate 25.3 TiB for an array with shape (12, 75, 331, 360, 180, 360) and data type float32

CHECK THE PROFILES

In [59]:
ID_open_list = [21,66,117,124,127,128] #,67,125

In [66]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

ax={}

leg_hdl = []
alpha_val = 0.2

i = 0

for kisf in tqdm(ID_open_list):

    ax[i] = f.add_subplot(2,3,i+1)
    

    if kisf not in [62,36]:
        if len(vv_mean_all.time_counter) > 1:
            for tt in vv_mean_all.time_counter:
    
                vvdata0 = vv_mean_all.sel(time_counter = tt, ID = kisf)
        else:
            vvdata0 = vv_mean_all.sel(ID = kisf).squeeze()
        
        ax[i].plot(vvdata0,-vvdata0.deptht, color='orange', alpha=alpha_val)



            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)


    ax[i].set_title(str(kisf))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

ax={}

leg_hdl = []
alpha_val = 0.2

i = 0

for kisf in tqdm(ID_open_list):

    ax[i] = f.add_subplot(2,3,i+1)
    

    if kisf not in [62,36]:
        for tt in S_mean_all.time_counter:

            S_data0 = S_mean_all.sel(time_counter = tt, ID = kisf)
            ax[i].plot(S_data0,-S_data0.z, color='orange', alpha=alpha_val)



            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)


    ax[i].set_title(str(kisf))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

PROPAGATE THEM

In [ ]:
new_T0 = T_init_0.copy()
new_S0 = S_init_0.copy()
for ID in tqdm(T_mean_all.ID):
    new_T0 = new_T0.where(isfmask_1deg != ID, T_mean_all.sel(ID=ID))
    new_S0 = new_S0.where(isfmask_1deg != ID, S_mean_all.sel(ID=ID))


In [ ]:
new_T0 = new_T0.where(isfmask_1deg != 67, T_mean_all.sel(ID=66).drop('ID'))
new_T0 = new_T0.where(isfmask_1deg != 125, T_mean_all.sel(ID=124).drop('ID'))
new_T0['lon'] = T_init_0['lon']
new_T0['lat'] = T_init_0['lat']
new_T0['nav_lev'] = T_init_0['nav_lev']


new_S0 = new_S0.where(isfmask_1deg != 67, S_mean_all.sel(ID=66).drop('ID'))
new_S0 = new_S0.where(isfmask_1deg != 125, S_mean_all.sel(ID=124).drop('ID'))
new_S0['lon'] = S_init_0['lon']
new_S0['lat'] = S_init_0['lat']
new_S0['nav_lev'] = S_init_0['nav_lev']

WRITE TO NETCDF

In [ ]:
new_T0.drop_vars('ID').to_netcdf(inputpath_interim + 'conservative_temperature_WOA13_decav_Reg1L75_clim_withisfcav.nc', unlimited_dims=['time_counter'])
new_S0.drop_vars('ID').to_netcdf(inputpath_interim + 'absolute_salinity_WOA13_decav_Reg1L75_clim_withisfcav.nc', unlimited_dims=['time_counter'])

In [ ]:
T_init_0

In [ ]:
new_T0.drop_vars('ID')

CHECK WHY THE FILES ARE SO LARGE

In [ ]:
T0 = xr.open_dataset(inputpath_interim + 'conservative_temperature_WOA13_decav_Reg1L75_clim_withisfcav.nc')

In [ ]:
T_init_0 - T0 

In [ ]:
T0['lon'] = T_init_0['lon']

In [ ]:
T0